In [1]:
# Trial 3의 최적 파라미터를 직접 저장하는 코드
import pickle

# 최적 파라미터 딕셔너리
best_params_trial3 = {
    'global_learning_rate': 0.05527306870639684,
    'global_n_estimators': 5822,
    'global_max_depth': 9,
    'global_subsample': 0.9544679438650066,
    'global_colsample_bytree': 0.8286782238457977,
    'global_min_child_weight': 8,
    'global_alpha': 2.9896290698185526,
    'ensemble_weight_type': 0.5257330598296284,
    'cdh_threshold': 20,
    'Hotel and Resort_max_depth': 4,
    'Hotel and Resort_subsample': 0.6710304947272696,
    'Hotel and Resort_colsample_bytree': 0.7225819434668705,
    'Hotel and Resort_min_child_weight': 7,
    'Hotel and Resort_alpha': 1.8559664363552197,
    'Commercial_max_depth': 12,
    'Commercial_subsample': 0.6392220434384972,
    'Commercial_colsample_bytree': 0.9515053077101255,
    'Commercial_min_child_weight': 1,
    'Commercial_alpha': 1.7726391394042025,
    'Hospital_max_depth': 12,
    'Hospital_subsample': 0.7834130786932383,
    'Hospital_colsample_bytree': 0.8747419906952096,
    'Hospital_min_child_weight': 10,
    'Hospital_alpha': 2.6696350178171677,
    'University_max_depth': 8,
    'University_subsample': 0.6486487772052955,
    'University_colsample_bytree': 0.9041585700584077,
    'University_min_child_weight': 4,
    'University_alpha': 1.6252188158037522,
    'Other Buildings_max_depth': 7,
    'Other Buildings_subsample': 0.8533171422821397,
    'Other Buildings_colsample_bytree': 0.7660582811424111,
    'Other Buildings_min_child_weight': 8,
    'Other Buildings_alpha': 1.7338394580293282,
    'Apartment_max_depth': 8,
    'Apartment_subsample': 0.7081977526024632,
    'Apartment_colsample_bytree': 0.909144720152893,
    'Apartment_min_child_weight': 5,
    'Apartment_alpha': 2.500036497364502,
    'Research Institute_max_depth': 4,
    'Research Institute_subsample': 0.84354910638235,
    'Research Institute_colsample_bytree': 0.8233897593306069,
    'Research Institute_min_child_weight': 6,
    'Research Institute_alpha': 2.779822733140019,
    'Department Store and Outlet_max_depth': 7,
    'Department Store and Outlet_subsample': 0.9688870153034745,
    'Department Store and Outlet_colsample_bytree': 0.6012249969270405,
    'Department Store and Outlet_min_child_weight': 9,
    'Department Store and Outlet_alpha': 2.92217446971382,
    'Data Center_max_depth': 5,
    'Data Center_subsample': 0.8827607655229703,
    'Data Center_colsample_bytree': 0.6658627945301703,
    'Data Center_min_child_weight': 5,
    'Data Center_alpha': 1.6775240395880826,
    'Public_max_depth': 6,
    'Public_subsample': 0.6043607703917003,
    'Public_colsample_bytree': 0.8508950613484505,
    'Public_min_child_weight': 6,
    'Public_alpha': 1.3198797950118193
}

# 최적 파라미터를 파일로 저장
def save_trial3_params():
    """Trial 3의 최적 파라미터를 저장"""
    with open('best_params.pkl', 'wb') as f:
        pickle.dump(best_params_trial3, f)
    print("Trial 3의 최적 파라미터가 'best_params.pkl'에 저장되었습니다.")
    print(f"저장된 파라미터 개수: {len(best_params_trial3)}")

# 저장 실행
save_trial3_params()

# 저장이 제대로 되었는지 확인
print("\n저장된 파라미터 확인:")
loaded_params = None
try:
    with open('best_params.pkl', 'rb') as f:
        loaded_params = pickle.load(f)
    print("✓ 파라미터 로딩 성공")
    print(f"Global learning rate: {loaded_params['global_learning_rate']:.6f}")
    print(f"Global n_estimators: {loaded_params['global_n_estimators']}")
    print(f"Ensemble weight type: {loaded_params['ensemble_weight_type']:.6f}")
    print(f"CDH threshold: {loaded_params['cdh_threshold']}")
except Exception as e:
    print(f"✗ 파라미터 로딩 실패: {e}")

Trial 3의 최적 파라미터가 'best_params.pkl'에 저장되었습니다.
저장된 파라미터 개수: 59

저장된 파라미터 확인:
✓ 파라미터 로딩 성공
Global learning rate: 0.055273
Global n_estimators: 5822
Ensemble weight type: 0.525733
CDH threshold: 20


In [ ]:
import optuna
from optuna.pruners import MedianPruner
import pickle
import os

def objective(trial):
    """Optuna 최적화 목적 함수"""
    
    # 글로벌 모델 파라미터 최적화
    global_params = {
        'learning_rate': trial.suggest_float('global_learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('global_n_estimators', 1000, 8000),
        'max_depth': trial.suggest_int('global_max_depth', 3, 12),
        'subsample': trial.suggest_float('global_subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('global_colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('global_min_child_weight', 1, 10),
        'alpha': trial.suggest_float('global_alpha', 1.0, 3.0),
        'random_state': RANDOM_SEED,
        'tree_method': 'gpu_hist'
    }
    
    # 앙상블 가중치 최적화
    ensemble_weight_type = trial.suggest_float('ensemble_weight_type', 0.3, 0.9)
    ensemble_weight_global = 1.0 - ensemble_weight_type
    
    # CDH 파라미터 최적화
    cdh_threshold = trial.suggest_int('cdh_threshold', 20, 30)
    
    # 타입별 모델 파라미터 최적화 (각 건물 타입별로)
    type_params = {}
    for building_type in type_list:
        type_params[building_type] = {
            'max_depth': trial.suggest_int(f'{building_type}_max_depth', 3, 12),
            'subsample': trial.suggest_float(f'{building_type}_subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float(f'{building_type}_colsample_bytree', 0.6, 1.0),
            'min_child_weight': trial.suggest_int(f'{building_type}_min_child_weight', 1, 10),
            'alpha': trial.suggest_float(f'{building_type}_alpha', 1.0, 3.0)
        }
    
    # 모델 학습 및 평가 (기존 로직과 동일하되 파라미터만 변경)
    kf = KFold(n_splits=7, shuffle=True, random_state=RANDOM_SEED)
    
    # 글로벌 모델 학습
    X_global = X.copy()
    Y_global = Y['power_consumption'].astype(float).copy()
    X_test_global = test_X.copy()
    
    Y_global = Y_global.replace([np.inf, -np.inf], np.nan)
    ok_global = Y_global.notna() & (Y_global >= 0)
    if not ok_global.all():
        X_global = X_global.loc[ok_global]
        Y_global = Y_global.loc[ok_global]
    
    X_global = pd.get_dummies(X_global, columns=['building_number', 'building_type'], drop_first=False)
    X_test_global = pd.get_dummies(X_test_global, columns=['building_number', 'building_type'], drop_first=False)
    X_global, X_test_global = X_global.align(X_test_global, join='outer', axis=1, fill_value=0)
    X_global = X_global.replace([np.inf, -np.inf], np.nan).fillna(0)
    X_test_global = X_test_global.replace([np.inf, -np.inf], np.nan).fillna(0)
    X_global = X_global.astype(np.float32)
    Y_global = Y_global.astype(np.float32)
    
    global_fold_smape = []
    global_pred = pd.DataFrame(index=Y_global.index.copy(), columns=['pred'])
    
    for train_index, valid_index in kf.split(X_global):
        X_train, X_valid = X_global.iloc[train_index], X_global.iloc[valid_index]
        Y_train, Y_valid = Y_global.iloc[train_index], Y_global.iloc[valid_index]
        
        Y_train_t = np.log1p(np.clip(Y_train, a_min=0, a_max=None))
        Y_valid_t = np.log1p(np.clip(Y_valid, a_min=0, a_max=None))
        
        evals = [(X_train, Y_train_t), (X_valid, Y_valid_t)]
        
        global_model = XGBRegressor(
            learning_rate=global_params['learning_rate'],
            n_estimators=global_params['n_estimators'],
            max_depth=global_params['max_depth'],
            random_state=global_params['random_state'],
            subsample=global_params['subsample'],
            colsample_bytree=global_params['colsample_bytree'],
            min_child_weight=global_params['min_child_weight'],
            objective=weighted_mse(global_params['alpha']),
            tree_method=global_params['tree_method'],
        )
        
        global_model.fit(
            X_train, Y_train_t,
            early_stopping_rounds=100,
            eval_metric=custom_smape,
            eval_set=evals,
            verbose=False
        )
        
        global_pred_t = global_model.predict(X_valid)
        global_pred_val = np.expm1(global_pred_t)
        global_pred.loc[Y_valid.index, 'pred'] = global_pred_val
        
        global_smape = smape(Y_valid.values, global_pred_val)
        global_fold_smape.append(global_smape)
    
    # 타입별 모델 학습
    type_pred_df = pd.DataFrame(columns=['pred'])
    
    for i in type_list:
        x = X[X.building_type == i].copy()
        y = Y[Y.building_type == i]['power_consumption'].astype(float).copy()
        
        y = y.replace([np.inf, -np.inf], np.nan)
        ok = y.notna() & (y >= 0)
        if not ok.all():
            x = x.loc[ok]
            y = y.loc[ok]
        
        x = pd.get_dummies(x, columns=['building_number'], drop_first=False)
        
        for df_ in (x,):
            if 'building_type' in df_.columns:
                df_.drop(columns=['building_type'], inplace=True)
        
        x = x.replace([np.inf, -np.inf], np.nan).fillna(0)
        x = x.astype(np.float32)
        y = y.astype(np.float32)
        
        type_fold_smape = []
        pred = pd.DataFrame(index=y.index.copy(), columns=['pred'])
        
        for train_index, valid_index in kf.split(x):
            X_train, X_valid = x.iloc[train_index], x.iloc[valid_index]
            Y_train, Y_valid = y.iloc[train_index], y.iloc[valid_index]
            
            Y_train_t = np.log1p(np.clip(Y_train, a_min=0, a_max=None))
            Y_valid_t = np.log1p(np.clip(Y_valid, a_min=0, a_max=None))
            
            evals = [(X_train, Y_train_t), (X_valid, Y_valid_t)]
            
            xgb_model = XGBRegressor(
                learning_rate=0.05,
                n_estimators=5000,
                max_depth=type_params[i]['max_depth'],
                random_state=RANDOM_SEED,
                subsample=type_params[i]['subsample'],
                colsample_bytree=type_params[i]['colsample_bytree'],
                min_child_weight=type_params[i]['min_child_weight'],
                objective=weighted_mse(type_params[i]['alpha']),
                tree_method='gpu_hist',
            )
            
            xgb_model.fit(
                X_train, Y_train_t,
                early_stopping_rounds=100,
                eval_metric=custom_smape,
                eval_set=evals,
                verbose=False
            )
            
            xgb_pred_t = xgb_model.predict(X_valid)
            xgb_pred = np.expm1(xgb_pred_t)
            pred.loc[Y_valid.index, 'pred'] = xgb_pred
            
            xgb_smape = smape(Y_valid.values, xgb_pred)
            type_fold_smape.append(xgb_smape)
        
        type_pred_df = pd.concat([type_pred_df, pred], axis=0)
    
    # 앙상블 계산
    ensemble_pred_df = pd.DataFrame(index=type_pred_df.index, columns=['pred'])
    for idx in type_pred_df.index:
        if idx in global_pred.index:
            type_pred = type_pred_df.loc[idx, 'pred']
            global_pred_val = global_pred.loc[idx, 'pred']
            ensemble_pred = (ensemble_weight_type * type_pred + 
                            ensemble_weight_global * global_pred_val)
            ensemble_pred_df.loc[idx, 'pred'] = ensemble_pred
        else:
            ensemble_pred_df.loc[idx, 'pred'] = type_pred_df.loc[idx, 'pred']
    
    # 최종 SMAPE 계산
    ensemble_score = smape(
        Y.loc[ensemble_pred_df.index, 'power_consumption'].values.astype(float),
        ensemble_pred_df['pred'].values.astype(float)
    )
    
    return ensemble_score

def optimize_parameters(n_trials=100):
    """파라미터 최적화 실행"""
    
    # Optuna study 생성
    study = optuna.create_study(
        direction='minimize',
        pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    )
    
    # 최적화 실행
    study.optimize(objective, n_trials=n_trials, timeout=None)
    
    # 최적 파라미터 저장
    best_params = study.best_params
    
    # 파라미터를 파일로 저장
    with open('best_params.pkl', 'wb') as f:
        pickle.dump(best_params, f)
    
    print(f"Best SMAPE: {study.best_value:.4f}")
    print("Best parameters saved to 'best_params.pkl'")
    
    return best_params



In [ ]:
# 파라미터 최적화 실행 (시간이 오래 걸리므로 필요할 때만 실행)
optimize_parameters(n_trials=100)  # 주석 해제하여 실행

# 이미 최적화가 완료되었다면 아래로 파라미터 확인
best_params = load_best_params()
if best_params:
    print("현재 저장된 최적 파라미터:")
    for key, value in best_params.items():
        print(f"{key}: {value}")